<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch16-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd

# RNN과 어텐션을 사용한 자연어 처리
- 자연어 문제를 위해 많이 사용하는 방법은 순환 신경망임
- 문자 단위 RNN: 문장에서 다음 글자를 예측
  - 상태가 없는 RNN: 각 반복에서 무작위하게 택한 텍스트의 일부분으로 학습하고 나머지 텍스트에서 어떤 정보도 사용하지 않음 
  - 상태가 있는 RNN: 훈련 반복 사이에 은닉 상태를 유지하고 중지된 곳에서 이어서 상태를 반영 -> 긴 패턴 학습 가능

- 단어 단위(시퀀스) RNN
- RNN 기반의 인코더-디코더 구조
- 어텐션 메커니즘
  - 각 타임 스텝에서 모델이 집중해야 할 입력 부분을 선택하도록 학습하는 신경망 구성 요소



## Char-RNN을 사용해 셰익스피어 문체 생성하기
- Char-RNN을 사용해서 한 번에 한 글자씩 새로운 텍스트를 생성할 수 있다.

### 훈련 데이터셋 만들기

In [3]:
url="https://homl.info/shakespeare"
filepath=keras.utils.get_file("shakespeare.txt", url)
with open(filepath) as f:
  text=f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [13]:
len(text)#=== 글자 수가 백만 개 이상

1115394

In [25]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

#### 모든 글자를 정수로 인코딩하기 -> 케라스의 Tokenizer클래스 아용

In [4]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)#=== 단어 수준이 아닌 글자 수준 인코딩
tokenizer.fit_on_texts(text)#=== 텍스트에 훈련

------
텍스트에 사용되는 모든 글자를 찾아 글자 ID에 매핑(ID는 1부터 시작)
<br><br>
- 간단한 테스트

In [5]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [6]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

- 전체 텍스트를 인코딩하여 각 글자를 ID로 나타내기
  - 1부터 시작하므로 -1해서 0부터 시작하는 걸로 바꿔보기

In [11]:
max_id =len(tokenizer.word_index)
max_id  #===== 즉, 고유 글자가 39개

39

In [7]:
[encoded] = np.array(tokenizer.texts_to_sequences([text]))-1

In [8]:
encoded

array([19,  5,  8, ..., 20, 26, 10])

### 순차 데이터셋을 나누는 방법
- 훈련/검증/테스트 세트가 <strong>중복되지 않도록</strong> 만드는 것이 매우 중요
  - 예를 들어 텍스트 처음 90%/5%/5%

- 시계열을 다룰 때는 보통 <strong>시간</strong>에 따라 나눈다.(안전하다)
  - ex)100개의 회사의 재정 건전성 관련 데이터 다루기 
    - 많은 회사들이 <strong>강하게 상호 연관</strong>되어 있을 가능성이 높다.
    - 훈련 세트와 테스트 세트에 상호 연관된 회사가 있다면 <strong>테스트 세트에서 측정한 일반 오차가 유용하지 않을 것임</strong>
    - => 따라서 시간에 따라 나누는 것이 일반적이고, 안전함

- 시계열이 <strong>충분히 안정적</strong>인지 꼭 확인해야한다.
  - 일반적으로는, 시계열이 '변하지 않는다'고 가정하지만, 일부는 그렇지 않음
    - ex) 금융 시장은 변덕스럽다. 트레이더가 패턴을 발견한 뒤 적용하려 하면 사라짐
  - 안정성 확인 방법: <strong>시간에 따라 검증 세트에 대한 모델의 오차 그려보기</strong>
    - 모델이 검증 세트 <strong>마지막보다 첫 부분에서 성능이 더 좋다면</strong> 이 시계열이 충분히 안정되지 않은 것일 수 있음
      - 이 경우, <strong>더 짧은 시간 간격</strong>으로 모델을 훈련하는 것이 좋다.

#### train: 텍스트의 처음 90%

In [9]:
dataset_size=tokenizer.document_count#== 전체 글자 개수
train_size=dataset_size*90//100 #=== 텍스트의 처음 90%
dataset= tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset

<TensorSliceDataset shapes: (), types: tf.int64>

### 순차 데이터를 윈도 여러 개로 자르기
- 해당 text는 백만 개 이상의 글자로 이루어진 시퀀스 하나
- 즉, 이대로 RNN으로 훈련 시키면, 백만 개의 층이 있는 심층 신경망과 같게 된다.
- 따라서 <strong>window()메서드</strong>를 사용해서 긴 문자열을 짧은 부분 문자열로 변환한다.
- 이 짧은 부분 문자열만큼만 역전파하는게 <strong>TBPTT(Truncated Backpropagation Through Time)</strong>
- BPTT vs TBPTT

  <img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdawXWw%2FbtqD2dngo5l%2FrvitZfPrq5bXlHPUfy07J0%2Fimg.png" width=50% />
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbiRY38%2FbtqD2bweMfV%2FXiaxKlF44miJP4IHATTfQk%2Fimg.png" width=50%/>

In [15]:
n_steps=100
window_length=n_steps+1
dataset=dataset.window(window_length,shift=1,drop_remainder=True) 

--------------
- window_length: 한 번에 얼마나 볼것인지(너무 짧게 만들면 긴 패턴을 학습할 수 없으므로 적절하게 조절해야함)
- shift: 윈도우를 얼마만큼 움직일 것인지(기본값=window 길이 )
- drop_remainder:  True로 하면 모든 윈도우가 동일한 개수의 글자를 포함하도록한다.
<br><br>

In [19]:
dataset= dataset.flat_map(lambda window: window.batch(window_length)) #=== 윈도우마다 batch(window_length)호출

--------
- flat_map()메서드를 통해서 데이터셋을 변환하고 평평하게 만든다
- ex) 기존 데이터셋(ds): {{1,2},{3,4,5,6}} <br>
flat_map(lambda ds: ds.batch(2)) ->   {[1,2],[3,4],[5,6]} : 텐서 2개를 가진 데이터셋
<br><br>

- 섞고, 타깃 값(마지막 1개 글자) 분리

In [22]:
batch_size=32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

- 입력 특성 원-핫 벡터
  - 고유한 글자 수가 적기 때문(39개)

In [23]:
dataset= dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)

In [24]:
dataset = dataset.prefetch(1)

### Char-RNN 모델 만들고 훈련하기